# Code for Creating the dataset for the NMF Simulation

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, imsave
import random
import os

plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

### Declaring helper functions

In [114]:
def atrophy_simulation(image, center, radius, intensity_drop):
    # Return the image denoised in an area, which is a disk of radius n and center.
    x,y = center
    for i in range(x-radius,x+radius):
        for j in range(y-radius,y+radius):
            if (x-i)**2 + (y-j)**2 <= radius**2:
                image[i,j] = image[i,j]*(1-intensity_drop)
    return image

### Create the noisy images

In [147]:
images = []
for i in range(1000):
    a = np.zeros((100,100))
    for (x,y), value in np.ndenumerate(a):
        a[x,y] = random.gauss(180, 15)
        if a[x,y] < 0:
            a[x,y] = 0
        elif a[x,y] > 255:
            a[x,y] = 255
    images.append(a)

### Remove the noise in specific areas

In [148]:
# R1 subtype: (~10% intensity reduction in R1 disc, 2 possible locations)            => 450 samples (225,225)
# R2 subtype: (~20% intensity reduction in R2 disc, overlapping with R1 subtype)     => 350 samples (175,175)
# R3 subtype: (~30% intensity reduction in R3 disc, overlapping with R1,R2 subtype)  => 200 samples (100,100)

R1 = 15
R2 = 10
R3 = 5

all_indexes = [x for x in range(1000)]
R1_A_indexes = random.sample(all_indexes, 225)
remaining_indexes = [x for x in all_indexes if x not in R1_A_indexes]
R1_B_indexes = random.sample(remaining_indexes, 225)
remaining_indexes = [x for x in remaining_indexes if x not in R1_B_indexes]
R1R2_A_indexes = random.sample(remaining_indexes, 175)
remaining_indexes = [x for x in remaining_indexes if x not in R1R2_A_indexes]
R1R2_B_indexes = random.sample(remaining_indexes, 175)
remaining_indexes = [x for x in remaining_indexes if x not in R1R2_B_indexes]
R1R2R3_A_indexes = random.sample(remaining_indexes, 100)
remaining_indexes = [x for x in remaining_indexes if x not in R1R2R3_A_indexes]
R1R2R3_B_indexes = random.sample(remaining_indexes, 100)


# Create R1 source A:
for i in R1_A_indexes:
    # Choose a random intensity between 8 and 11 %:
    atroph = random.choice([0.08, 0.09, 0.1 ,0.11])
    r1_a = atrophy_simulation(images[i], (25,25), R1, atroph)
    imsave(str(i)+"_R1_A_"+str(int(atroph*100))+".jpg", r1_a, cmap = 'gray', vmin=0, vmax=255)



# Create R1 source B:
for i in R1_B_indexes:
    # Choose a random intensity between 8 and 11 %:
    atroph = random.choice([0.08, 0.09, 0.1 ,0.11])
    r1_b = atrophy_simulation(images[i], (60,60), R1, atroph)
    imsave(str(i)+"_R1_B_"+str(int(atroph*100))+".jpg", r1_b, cmap = 'gray', vmin=0, vmax=255)



# Create R1R2 source A:
for i in R1R2_A_indexes:
    # Choose a random intensity between 8 and 11 %:
    atroph1 = random.choice([0.08, 0.09, 0.1 ,0.11])
    r1_a = atrophy_simulation(images[i], (25,25), R1, atroph1)
    # Choose a random intensity between 18 and 21 %:
    atroph2 = random.choice([0.18, 0.19, 0.2 ,0.21])
    r1r2_a = atrophy_simulation(r1_a, (25,25), R2, atroph2 - atroph1)
    imsave(str(i)+"_R1R2_A_"+str(int(atroph1*100))+"_"+str(int(atroph2*100))+".jpg", r1r2_a, cmap = 'gray', vmin=0, vmax=255)



# Create R1R2 source B:
for i in R1R2_B_indexes:
    # Choose a random intensity between 8 and 11 %:
    atroph1 = random.choice([0.08, 0.09, 0.1 ,0.11])
    r1_b = atrophy_simulation(images[i], (60,60), R1, atroph1)
    # Choose a random intensity between 18 and 21 %:
    atroph2 = random.choice([0.18, 0.19, 0.2 ,0.21])
    r1r2_b = atrophy_simulation(r1_b, (60,60), R2, atroph2 - atroph1)
    imsave(str(i)+"_R1R2_B_"+str(int(atroph1*100))+"_"+str(int(atroph2*100))+".jpg", r1r2_b, cmap = 'gray', vmin=0, vmax=255)



# Create R1R2R3 source A:
for i in R1R2R3_A_indexes:
    # Choose a random intensity between 8 and 11 %:
    atroph1 = random.choice([0.08, 0.09, 0.1 ,0.11])
    r1_a = atrophy_simulation(images[i], (25,25), R1, atroph1)
    # Choose a random intensity between 18 and 21 %:
    atroph2 = random.choice([0.18, 0.19, 0.2 ,0.21])
    r1r2_a = atrophy_simulation(r1_a, (25,25), R2, atroph2 - atroph1)
    # Choose a random intensity between 28 and 31 %:
    atroph3 = random.choice([0.28, 0.29, 0.3 ,0.31])
    r1r2r3_a = atrophy_simulation(r1r2_a, (25,25), R3, atroph3 - atroph2 + atroph1)    
    imsave(str(i)+"_R1R2R3_A_"+str(int(atroph1*100))+"_"+str(int(atroph2*100))+"_"+str(int(atroph3*100))+".jpg", r1r2r3_a, cmap = 'gray', vmin=0, vmax=255)



# Create R1R2R3 source B:
for i in R1R2R3_B_indexes:
    # Choose a random intensity between 8 and 11 %:
    atroph1 = random.choice([0.08, 0.09, 0.1 ,0.11])
    r1_b = atrophy_simulation(images[i], (60,60), R1, atroph1)
    # Choose a random intensity between 18 and 21 %:
    atroph2 = random.choice([0.18, 0.19, 0.2 ,0.21])
    r1r2_b = atrophy_simulation(r1_b, (60,60), R2, atroph2)
    # Choose a random intensity between 28 and 31 %:
    atroph3 = random.choice([0.28, 0.29, 0.3 ,0.31])
    r1r2r3_b = atrophy_simulation(r1r2_b, (60,60), R3, atroph3 - atroph2 + atroph1)
    imsave(str(i)+"_R1R2R3_B_"+str(int(atroph1*100))+"_"+str(int(atroph2*100))+"_"+str(int(atroph3*100))+".jpg", r1r2r3_b, cmap = 'gray', vmin=0, vmax=255)